# Intro to MLOps using ZenML

## 🌍 Overview

This repository is a minimalistic MLOps project intended as a starting point to learn how to put ML workflows in production.

Within this notebook we will show you how simple it is to switch from running code locally to running it remotely. You will then be able to explore all the metadata of your run in the ZenML Dashboard.

<img src=".assets/Overview.png" width="50%" alt="Quickstart Overview">

Follow along this notebook to understand how you can use ZenML to productionalize your ML workflows!

## Run on Colab

You can use Google Colab to run this notebook, no local installation
required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/quickstart.ipynb)

# 👶 Step 0. Install Requirements

Let's install ZenML and all requirement to get started.

In [1]:
!pip install "zenml[server]"

  Using cached zenml-0.63.0-py3-none-any.whl.metadata (21 kB)
  Using cached alembic-1.8.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached bcrypt-4.0.1-cp36-abi3-manylinux_2_28_x86_64.whl.metadata (9.0 kB)
  Using cached click-8.1.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached click_params-0.3.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached httplib2-0.19.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached ipinfo-5.0.1-py3-none-any.whl.metadata (648 bytes)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached orjson-3.10.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (50

In [2]:
from zenml.environment import Environment

# In case we are in a google colab, clone all additional relevant files
if Environment.in_google_colab():
    # Pull required modules from this example
    !git clone -b main https://github.com/zenml-io/zenml
    !cp -r zenml/examples/quickstart/* .
    !rm -rf zenml

!pip install -r requirements.txt

  Using cached pyarrow-17.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
  Using cached torch-2.4.0-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
  Using cached xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached regex-2024.7.24-cp311-cp311-manylinux_2_17_x86_

In [5]:
# Restart Kernel to ensure all libraries are properly loaded
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}


Please wait for the installation to complete before running subsequent cells. At
the end of the installation, the notebook kernel will restart.

## ☁️ Step 1: Connect to your ZenML Server
To run this quickstart you need to connect to a ZenML Server. You can deploy it [yourself on your own infrastructure](https://docs.zenml.io/getting-started/deploying-zenml) or try it out for free, no credit-card required in our [ZenML Pro managed service](https://zenml.io/pro).

In [3]:
zenml_server_url = "INSERT_YOUR_SERVER_URL_HERE"  # in the form "https://URL_TO_SERVER"

!zenml connect --url $zenml_server_url

Connecting to: 'INSERT_YOUR_SERVER_URL_HERE'...
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/alexej/.pyenv/versions/clean3/bin/zenml:8 in <module>                  │
│                                                                              │
│   5 from zenml.cli.cli import cli                                            │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│   9                                                                          │
│                                                                              │
│ /home/alexej/.pyenv/versions/3.11.9/envs/clean3/lib/python3.11/site-packages │
│ /click/core.py:1130 in __call__                                              │
│                                                            

In [2]:
# Initialize ZenML and define the root for imports and docker builds
!zenml init

Found existing ZenML repository at path 
'/home/alexej/PycharmProjects/zenml/examples/quickstart'.
⠋ Initializing ZenML repository at 
/home/alexej/PycharmProjects/zenml/examples/quickstart.
⠋ Initializing ZenML repository at 
/home/alexej/PycharmProjects/zenml/examples/quickstart.



## 🥇 Step 2: Build and run your first pipeline

In this quickstart we'll be working with a small dataset of sentences in old english paired with more modern formulations. The task is a text-to-text transformation.

When you're getting started with a machine learning problem you'll want to break down your code into distinct functions that load your data, bring it into the correct shape and finally produce a model. This is the experimentation phase where we try to massage our data into the right format and feed it into our model training.

<img src=".assets/Experiment.png" width="50%" alt="Experiment">

In [5]:
from zenml import step
from datasets import Dataset
from typing_extensions import Annotated


@step
def load_data() -> Annotated[Dataset, "raw_dataset"]:
    """Load and prepare the dataset."""

    def read_data(file_path):
        inputs = []
        targets = []

        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                old, modern = line.strip().split("|")
                inputs.append(f"Translate Old English to Modern English: {old}")
                targets.append(modern)

        return {"input": inputs, "target": targets}

    # Assuming your file is named 'translations.txt'
    data = read_data("translations.txt")
    return Dataset.from_dict(data)

ZenML is built in a way that allows you to experiment with your data and build
your pipelines one step at a time.  If you want to call this function to see how it
works, you can just call it directly. Here we take a look at the first few rows
of your training dataset.

In [11]:
dataset = load_data()
dataset[0]

{'input': "translate Old English to Modern English: Shall I compare thee to a summer's day?",
 'target': 'Should I compare you to a summer day?'}

Everything looks as we'd expect and the input/output pair looks to be in the right format 🥳.

For the sake of this quickstart we have prepared a few steps in the steps-directory. We'll now connect these together into a pipeline. To do this simply plug multiple steps together through their inputs and outputs. Then just add the `@pipeline` decorator to the function that connects the steps.

In [14]:
from zenml import pipeline
from zenml.client import Client

from steps import load_data, tokenize_data, train_model, evaluate_model, test_random_sentences
from steps.model_trainer import T5_Model

# Initialize the ZenML client to fetch objects from the ZenML Server
client = Client()

Client().activate_stack("default") # We will start by using the default stack which is local

@pipeline(enable_cache=True)
def english_translation_pipeline(
    model_type: T5_Model,
    num_train_epochs: int,
    per_device_train_batch_size: int,
    gradient_accumulation_steps: int,
    dataloader_num_workers: int,
):
    """Define a pipeline that connects the steps."""
    dataset = load_data()
    tokenized_dataset = tokenize_data(dataset)
    model, tokenizer = train_model(
        tokenized_dataset,
        model_type,
        num_train_epochs,
        per_device_train_batch_size,
        gradient_accumulation_steps,
        dataloader_num_workers,
    )
    evaluate_model(model, tokenized_dataset)
    test_random_sentences(model, tokenizer)

We're ready to run the pipeline now, which we can do just as with the step - by calling the
pipeline function itself:

In [15]:
# Run the pipeline and configure some parameters at runtime
pipeline_run = english_translation_pipeline(
    model_type="t5-small",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    dataloader_num_workers=4
)

Initiating a new run for the pipeline: english_translation_pipeline.
Executing a new run.
Caching is disabled by default for english_translation_pipeline.
Using user: alexej@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Dashboard URL: https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/8a462fb6-b1fe-48df-9677-edc76bc8352d/runs/6fefd974-20bf-4cde-a7e1-977b8617c6d6
Step load_data has started.
No materializer is registered for type <class 'datasets.arrow_dataset.Dataset'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'datasets.arrow_dataset.Dataset'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
Step load_data has finished in 1.122s.
Step load_data c

Map:   0%|          | 0/456 [00:00<?, ? examples/s]

No materializer is registered for type <class 'datasets.arrow_dataset.Dataset'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'datasets.arrow_dataset.Dataset'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
Step tokenize_data has finished in 1.795s.
Step tokenize_data completed successfully.
Caching disabled explicitly for train_model.
Step train_model has started.


Step,Training Loss
10,10.445600
20,6.838300
30,4.886000
40,3.074800
50,2.046200
60,1.572100
70,1.393100
80,1.238600
90,1.051400
100,0.995100


/home/alexej/.pyenv/versions/3.11.9/envs/clean3/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test translation: Translat Translate Old English Translate English Old
Model training completed and saved.
Step train_model has finished in 6m5s.
Step train_model completed successfully.
Step evaluate_model has started.
Average loss on the dataset: 0.40942612290382385
Step evaluate_model has finished in 30.768s.
Step evaluate_model completed successfully.
Step test_random_sentences has started.
/home/alexej/.pyenv/versions/3.11.9/envs/clean3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should 

As you can see the pipeline has run succesfully. It also printed out some examples - however it seems the model did not perform well.

Lets check have a look at this Pipeline in the ZenML Dashboard. You can find the URL above in the logs. 

We can also fetch the pipeline from the server and view the results directly in the notebook:

In [ ]:
client = Client()
run = client.get_pipeline("english_translation_pipeline").last_run
print(run.name)

We can also access the trained model directly by accessing the run. The cool thing here is, this direct access is gonna be available not just now, within this notebook session, but at any later point when you or your colleaugues might need it.

In [ ]:
# load the model object
model = run.steps["train_model"].outputs["model"].load()
tokenizer = run.steps["train_model"].outputs["tokenizer"].load()

With these in hand we can now play around with the model directly and try out some examples ourselves:

In [ ]:
test_text = "I do desire we may be better strangers"

input_ids = tokenizer(
    test_text,
    return_tensors="pt",
    max_length=128,
    truncation=True,
    padding="max_length",
).input_ids

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=128,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded_output)

## Lets recap what we've done so far

1) We have created a pipeline that takes in a dataset and trains a small Model on it
2) This pipeline is broken down in such a way that we can easily iterate on the individual parts without breaking the whole

As expected, the performance of this model is not good. To train a model that can solve our task well, we wopuld have to train a larger model. For this, we'll need to move onto the next section.

# ⌚ Step 3: Scale it up in the cloud

The model we have trained on our local machine is proof that our pipeline works. However, we want to train a much more powerful model. For this we'll need to scale onto more powerful machines.

So lets take this to the next level and run the pipeline in the environment of your choice. In ZenML we use the word "stack" to describe the environment for a pipline run. A stack consists of different components, however you'll only really need to care about the compute and data storage for this example.

For you to be able to try this step, you will need to have acess to some cloud compute and cloud storage somewhere (aws, gcp, azure, etc...). ZenML wrapps around all the major cloud providers and orchestration tools and lets you easily plug your code into them.

To do this lets head over to the Stack section of your ZenML Dashboard. Here you'll be able to either connect to an existing or deploy a new environment. Choose on of the options presented to you there and come back when you have a stack ready to go. 

<img src=".assets/stack_creation.png" width="50%" alt="Pipelines Overview">

## GCP

In [ ]:
!zenml integration install gcp -y

from zenml.client import Client
from zenml.integrations.gcp.flavors.vertex_orchestrator_flavor import VertexOrchestratorSettings

# Set the name of your stack here
stack_name = "alexej-gcp-quickstart-stack2"

Client().activate_stack(stack_name)

configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "resources": ResourceSettings(memory="32GB"),
        "orchestrator.vertex": VertexOrchestratorSettings(node_selector_constraint=("cloud.google.com/gke-accelerator", "NVIDIA_TESLA_P4"))
    }
)

## AWS

In [ ]:
!zenml integration install aws s3 -y

from zenml.client import Client
from zenml.integrations.aws.flavors.sagemaker_orchestrator_flavor import SagemakerOrchestratorSettings

# Set the name of your stack here
stack_name = "alexej-aws-quickstart-stack"

Client().activate_stack(stack_name)

configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "resources": ResourceSettings(memory="32GB"),
        "orchestrator.sagemaker": SagemakerOrchestratorSettings(instance_type="ml.p2.xlarge")
    }
)

## Azure

In [ ]:
!zenml integration install azure -y

from zenml.client import Client
from zenml.integrations.skypilot.flavors.skypilot_orchestrator_base_vm_config import SkypilotBaseOrchestratorSettings

# Set the name of your stack here
stack_name = "azure-quickstart-stack"

Client().activate_stack(stack_name)

configured_english_translation_pipeline = english_translation_pipeline.with_options(
    settings={
        "docker": DockerSettings(
            parent_image="pytorch/pytorch:2.4.0-cuda11.8-cudnn9-runtime",
            requirements=["zenml==0.63.0","pyarrow","datasets","transformers[torch]","sentencepiece"],
            environment={"ZENML_DISABLE_STEP_LOGS_STORAGE": True}
        ),
        "orchestrator.sagemaker": SkypilotBaseOrchestratorSettings(accelerators='V100', memory="32+", cpus="8+")
    }
)

## Ready to launch

We now have configured zenml to use your very own cloud infrastructure for the next pipeline run, lets see this in action by running the pipeline again on the smaller t5 model (`t5_small`).

Note: The whole process may take a bit longer the first time around, as your pipeline code needs to be built into docker containers to be run in the orchestration environment of your stack. Any consecutive run of the pipeline, even with different parameters set, will not take as long again thanks to docker caching.

In [ ]:
pipeline_run = configured_english_translation_pipeline(
    model_type="t5-small",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    dataloader_num_workers=0
)

Once this pipeline has run through fully, lets check how this model on our example:

## Now its Up to you

...

## Congratulations!

You're a legit MLOps engineer now! You have created a training pipeline and you
have deployed it into a production-ready environment with the compute of your 
choice. You also have gotten a hang of the ZenML Dashboard.

## Further exploration

This was just the tip of the iceberg of what ZenML can do; check out the [**docs**](https://docs.zenml.io/) to learn more
about the capabilities of ZenML. For example, you might want to:

- [Deploy ZenML](https://docs.zenml.io/user-guide/production-guide/connect-deployed-zenml) to collaborate with your colleagues.
- Run the same pipeline on a [cloud MLOps stack in production](https://docs.zenml.io/user-guide/production-guide/cloud-stack).
- Track your metrics in an experiment tracker like [MLflow](https://docs.zenml.io/stacks-and-components/component-guide/experiment-trackers/mlflow).

## What next?

* If you have questions or feedback... join our [**Slack Community**](https://zenml.io/slack) and become part of the ZenML family!
* If you want to quickly get started with ZenML, check out [ZenML Pro](https://zenml.io/pro).